## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('../Data/ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('../Data/ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [13]:
from collections import Counter

In [14]:
sentences_ted_all = [i for row in sentences_ted for i in row]
c = Counter(sentences_ted_all)
counts_ted_top1000_cunter = c.most_common(1000)

In [15]:
counts_ted_top1000_cunter[:5]


[('the', 207748),
 ('and', 149305),
 ('to', 125169),
 ('of', 114818),
 ('a', 105399)]

In [16]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
vec = CountVectorizer()
X = vec.fit_transform(sentences_strings_ted)  # 不同点不需要分词 直接用句子处理

In [18]:
counts_ted_top40_index = np.array(np.argsort(-X.sum(axis=0))).squeeze()[:40]
counts_ted_top40_word = np.array(vec.get_feature_names())[counts_ted_top40_index]
counts_ted_top40 = np.array(-1 * np.sort(-X.sum(axis=0))).squeeze()[:40]

print(counts_ted_top40_word)

['the' 'and' 'to' 'of' 'that' 'in' 'it' 'you' 'we' 'is' 'this' 'so' 'they'
 'was' 'for' 'are' 'have' 'but' 'what' 'on' 'with' 'can' 'about' 'there'
 'be' 'as' 'at' 'all' 'not' 'do' 'my' 'one' 're' 'people' 'like' 'if'
 'from' 'now' 'our' 'he']


In [19]:
counts_ted_top1000_index = np.array(np.argsort(-X.sum(axis=0))).squeeze()[:1000]
counts_ted_top1000_word = np.array(vec.get_feature_names())[counts_ted_top1000_index]
counts_ted_top1000 = np.array(-1 * np.sort(-X.sum(axis=0))).squeeze()[:1000]

Plot distribution of top-1000 words

In [20]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The normed argument is ignored when density is provided. In future passing both will result in an error.
  """Entry point for launching an IPython kernel.


### Part 3: Train Word2Vec

In [21]:
from gensim.models import Word2Vec,FastText
from gensim import corpora

In [22]:
model_ted = Word2Vec(sentences=sentences_ted, size=100, window=5, min_count=10, workers=4, sg=0)
dictionary_ted = corpora.Dictionary(sentences_ted)
print(len(dictionary_ted), dictionary_ted)

54454 Dictionary(54454 unique tokens: ['are', 'companies', 'do', 'fail', 'here']...)


In [23]:
model_fas = FastText(sentences=sentences_ted, size=100, window=5, min_count=10, workers=4, sg=1)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [24]:
model_ted.wv.most_similar("man")

[('woman', 0.8279821276664734),
 ('guy', 0.8109195232391357),
 ('boy', 0.7652816772460938),
 ('lady', 0.7546353936195374),
 ('girl', 0.7393965721130371),
 ('gentleman', 0.7377394437789917),
 ('soldier', 0.7244832515716553),
 ('poet', 0.6874878406524658),
 ('kid', 0.6795066595077515),
 ('king', 0.660785436630249)]

In [25]:
model_fas.wv.most_similar('man')

[('batman', 0.8157356977462769),
 ('woman', 0.7853835821151733),
 ('foreman', 0.775435209274292),
 ('shaman', 0.7752416133880615),
 ('fireman', 0.7634155750274658),
 ('guy', 0.7382100224494934),
 ('anderson', 0.7258508205413818),
 ('goldman', 0.7238233089447021),
 ('roman', 0.7079381942749023),
 ('ottoman', 0.7032468318939209)]

In [26]:
model_ted.wv.most_similar("computer")

[('machine', 0.735552191734314),
 ('software', 0.7269195318222046),
 ('robot', 0.6898480653762817),
 ('3d', 0.6603951454162598),
 ('device', 0.6594791412353516),
 ('chip', 0.6557748913764954),
 ('interface', 0.6448720097541809),
 ('simulation', 0.6298094987869263),
 ('desktop', 0.628256618976593),
 ('video', 0.6231306791305542)]

In [27]:
model_fas.wv.most_similar("Gastroenteritis")

[('arthritis', 0.8259387016296387),
 ('h1n1', 0.7725438475608826),
 ('adolescent', 0.7597124576568604),
 ('h5n1', 0.7475234270095825),
 ('ghraib', 0.7466444969177246),
 ('cdc', 0.7410764694213867),
 ('adolescents', 0.7367986440658569),
 ('mitochondrial', 0.7339369058609009),
 ('antiangiogenic', 0.7272385358810425),
 ('astrolabe', 0.7265154123306274)]

In [28]:
words_top_ted = counts_ted_top1000_word

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [29]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [30]:
from sklearn.manifold import TSNE

In [31]:
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [32]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [33]:
if not os.path.isfile('../Data/wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [34]:
with zipfile.ZipFile('../Data/wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [35]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z0-9]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [36]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [37]:
wiki_ted = []
for wiki_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", wiki_str.lower()).split()
    # tokens = re.sub(r'[{}]+'.format('-!:,.;?"'), " ", sent_str.lower()).split()
    wiki_ted.append(tokens)

In [38]:
model_wiki = Word2Vec(sentences=wiki_ted, size=100, window=5, min_count=10, workers=4, sg=0)
dictionary_wiki = corpora.Dictionary(wiki_ted)
print(len(dictionary_wiki), dictionary_wiki)

244747 Dictionary(244747 unique tokens: ['16', 'a', 'africa', 'atlantic', 'coast']...)


In [39]:
wiki = CountVectorizer()
Y = wiki.fit_transform(sentences_wiki)
wiki_ted_top1000_index = np.array(np.argsort(-Y.sum(axis=0))).squeeze()[:1000]
wiki_ted_top1000_word = np.array(wiki.get_feature_names())[wiki_ted_top1000_index]
wiki_ted_top1000 = np.array(-1 * np.sort(-Y.sum(axis=0))).squeeze()[:1000]

#### t-SNE visualization

In [40]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_wiki = wiki_ted_top1000_word
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

In [42]:
from sklearn.cluster import KMeans

In [43]:
clustering = KMeans(n_clusters=100)  # 聚几个类

clustering.fit(words_top_vec_wiki)
clu_labels = clustering.labels_  # 0-100 个标签

print(wiki_ted_top1000_word[clu_labels == 0])
print(wiki_ted_top1000_word[clu_labels == 1])

['king' 'ii' 'royal' 'queen']
['10' '12' '20' '15' '30' '11' '18' '16' '14' '13' '25' '17' '24' '21'
 '19' '22' '23' '28' '27' '26' '29' '31']


In [44]:
colors = ['#%d%d%d' % (int(2.55 * r), int(1.5 * r), int(1 * r)) for r in clu_labels]


In [45]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

clustering_source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:, 0],
                                               x2=words_top_wiki_tsne[:, 1],
                                               names=words_top_wiki,
                                               colors=colors))

p.scatter(x="x1", y="x2", size=8, source=clustering_source, fill_color='colors')

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=clustering_source, text_align='center')
p.add_layout(labels)

show(p)